# Conexão com banco

In [130]:
import psycopg2
import pandas as pd
import psycopg2
from IPython.display import display

host = "silly.db.elephantsql.com"
port = "5432"
database = "ljppladj"
user = "ljppladj"
password = "JFgtpa3EjnB6zOJ75ejGn_gUOl8-dy3j"

try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar: {e}")


Conexão bem-sucedida!


# Populando dataframe

In [131]:
# Crie um cursor para executar comandos SQL no banco de dados
cursor = conn.cursor()

# Execute a consulta SQL para juntar as tabelas e obter os dados desejados
cursor.execute("""
  SELECT 
        candidates.cnd_id, 
        candidates.cnd_name, 
        candidates.cnd_current_profession, 
        candidates.cnd_experience, 
        skl_skill.skl_name as skill,
        knw_knowledge.knw_name as knowledge,
        att_attitude.att_name as attitude
    FROM candidates
    LEFT JOIN candidate_skill ON candidates.cnd_id = candidate_skill.cnd_id
    LEFT JOIN skl_skill ON candidate_skill.knw_id = skl_skill.skl_id
    LEFT JOIN knw_knowledge ON skl_skill.skl_id = knw_knowledge.knw_id
    LEFT JOIN candidate_attitude ON candidates.cnd_id = candidate_attitude.cnd_id
    LEFT JOIN att_attitude ON candidate_attitude.knw_id = att_attitude.att_id

""")

df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Remover duplicatas mantendo apenas as colunas relevantes
df_skills = df[['cnd_id', 'cnd_name', 'cnd_experience', 'skill']].drop_duplicates()
df_knowledge = df[['cnd_id', 'cnd_name', 'cnd_experience', 'knowledge']].drop_duplicates()
df_attitude = df[['cnd_id', 'cnd_name', 'cnd_experience', 'attitude']].drop_duplicates()

# display(df_skills)
# display(df_knowledge)
# display(df_attitude)

cursor.close()

# Tratamento de dados

In [137]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split


desenvolvedor = {
    'profissao': 'Desenvolvedor Back-End',
    'habilidades': [
        'Python', 
        'Java', 
        'Node.js', 
        'MySQL',
        'PostgreSQL'
    ],
    'conhecimentos': [
        'Manipulação de Dados',
        'Conhecimento em SQL',
    ],
    'atitudes': [
        'Lógica de Programação',
        'Resolução de Problemas',
        'Criatividade',
    ]
}


habilidades = desenvolvedor['habilidades']
conhecimentos = desenvolvedor['conhecimentos']
atitudes = desenvolvedor['atitudes']

df_skills['tem_habilidade'] = 0
df_knowledge['tem_conhecimento'] = 0
df_attitude['tem_atitude'] = 0

# Iterar sobre as habilidades e marcar como 1 se presente na coluna 'skill'
# Para as habilidades
for habilidade in habilidades:
    df_skills['tem_habilidade'] = (df_skills['tem_habilidade'] | df_skills['skill'].fillna('').str.contains(habilidade, case=False)).astype(int)

# Para os conhecimentos
for conhecimento in conhecimentos:
    df_knowledge['tem_conhecimento'] = (df_knowledge['tem_conhecimento'] | df_knowledge['knowledge'].fillna('').str.contains(conhecimento, case=False)).astype(int)

# Para as atitudes
for atitude in atitudes:
    df_attitude['tem_atitude'] = (df_attitude['tem_atitude'] | df_attitude['attitude'].fillna('').str.contains(atitude, case=False)).astype(int)


# display(df_skills)
# display(df_knowledge)
# display(df_attitude)


# Treinamento dos dados

In [133]:
# dfCandidatos['CHA'] = dfCandidatos['CHA'] + ' ' + cha_vaga

# X = dfCandidatos[['CHA', 'cha_vaga']]
# y = dfCandidatos['Rotulo']
# X_combined = X['CHA'] + ' ' + X['cha_vaga']

# teste = dfCandidatos['skills']
# X_train, X_test, y_train, y_test = train_test_split(teste, y, train_size=0.33, random_state=42)

# # Modelo de classificação
# model = make_pipeline(CountVectorizer(), MultinomialNB())
# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)
# # print(metrics.accur   acy_score(y_test, y_pred))

# # Obtendo as probabilidades previstas pelo modelo para a classe 'apto'
# probabilidades_apto = model.predict_proba(X_test)[:, 0]

# # # Adicionando as probabilidades ao DataFrame
# dfCandidatos.loc[X_test.index, 'Probabilidade_Apto'] = probabilidades_apto

# # # Ordenando os candidatos pela probabilidade de serem 'apto'
# candidatos_ordenados = dfCandidatos.sort_values(by='Probabilidade_Apto', ascending=False)

# # # Selecionando os 3 candidatos mais prováveis
# top_candidatos = candidatos_ordenados.head(6)

# accuracy = model.score(X_test, y_test)

# display(top_candidatos)

KeyError: 'CHA'